<a href="https://colab.research.google.com/github/hanmilLee/for_ds9/blob/main/Day2_torch_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day2
## Speech Command Recognition baseline with Torch


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install colab_ssh on google colab (vscode에서 ssh 실행할 목적)
!pip install colab_ssh --upgrade 

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared 
launch_ssh_cloudflared(password="asd")

# Optional: if you want to clone a github repository
init_git_cloudflared('https://github.com/hanmilLee/for_ds9.git')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Successfully cloned the repository in ./for_ds9


In [1]:
! git config --global user.name "hanmilLee"
! git config --global user.email 94hanmil@gmail.com


In [2]:
%cd /content

/content


패키지 설치

In [3]:
!pip install torchaudio torchinfo
!pip install einops
# !pip install pyroomacoustics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
import torch.fft
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from torchinfo import summary

from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torchaudio.datasets import SPEECHCOMMANDS

import numpy as np
import scipy
from scipy import signal

import soundfile as sf

import librosa
import librosa.display

import matplotlib
import matplotlib.pyplot as plt
import IPython.display as ipd

# import pyroomacoustics as pra

from einops import rearrange

import io
import os
import math
import glob
from tqdm import tqdm
from collections import OrderedDict


GPU 동작 확인

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Public dataset 경로 \
https://drive.google.com/drive/folders/1CJuVPfiYsI4v1BK-SakT7frf8Nv-p-bn

In [6]:
!ls drive/MyDrive/public_set

npy  wav


In [7]:
!ls drive/MyDrive/public_set/npy

0  1  2


Train / Valid set Download

In [8]:
train_set = torchaudio.datasets.SPEECHCOMMANDS(root = '', download = True, subset = "training")
valid_set = torchaudio.datasets.SPEECHCOMMANDS(root = '', download = True, subset = "validation")

waveform, sample_rate, label, speaker_id, utterance_number = train_set[0]
print(train_set[0])

  0%|          | 0.00/2.26G [00:00<?, ?B/s]

(tensor([[-0.0658, -0.0709, -0.0753,  ..., -0.0700, -0.0731, -0.0704]]), 16000, 'backward', '0165e0e8', 0)


In [9]:
labels = sorted(list(set(datapoint[2] for datapoint in valid_set)))

def label_to_index(word):
    return torch.tensor(labels.index(word))
    
print(' '.join(['{} {}'.format(label_to_index(label), label) for label in labels]))

0 backward 1 bed 2 bird 3 cat 4 dog 5 down 6 eight 7 five 8 follow 9 forward 10 four 11 go 12 happy 13 house 14 learn 15 left 16 marvin 17 nine 18 no 19 off 20 on 21 one 22 right 23 seven 24 sheila 25 six 26 stop 27 three 28 tree 29 two 30 up 31 visual 32 wow 33 yes 34 zero


Torch Dataset

In [10]:
# Todo: 개인 구글 드라이브에 노이즈 파일 올려놓고 조원들에게 구글드라이브 바로가기 경로 수정하도록~

# 노이즈 불러오기data_dir = 'drive/MyDrive/tutorial_1_samples'
noise_data_dir = 'drive/MyDrive/tutorial_2_noises'
noise, sample_rate = torchaudio.load(os.path.join(noise_data_dir, 'exercise_bike.wav'))


In [24]:
class Custom_CommandDataset(Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.command_dataset = dataset

    def __getitem__(self, idx):
        raw_wav = self.command_dataset[idx][0][0]
        noise, sample_rate = torchaudio.load(os.path.join(noise_data_dir, 'exercise_bike.wav'))
        raw_wav_pad = np.pad(raw_wav, (0, 16000 - raw_wav.shape[-1]))
        # 여기서 데이터 처리 (CPU에서 진행된다. 메모리에 파일 올려놓았다가 여기서 처리하는게 효율적일 것이다.)

        
        

        noise = noise[:, 0:16000]


        
        noise_power = noise.norm(p=2)
    
        
        noise_power = noise_power.squeeze()
        print(noise_power.squeeze().shape)
    
        

      
        snr = 10 ** (3 / 10)
        scale = snr * noise_power / raw_wav_pad
        noisy_raw_wav_pad = (scale * raw_wav_pad + noise) / 2


        label_word = self.command_dataset[idx][2]
        label_index = label_to_index(label_word)
        print(raw_wav_pad.shape,noisy_raw_wav_pad.shape)



        return noisy_raw_wav_pad, label_index
        

    def __len__(self):
        return len(self.command_dataset)

train_set_custom = Custom_CommandDataset(train_set)
valid_set_custom = Custom_CommandDataset(valid_set)

In [27]:
import numpy as np

# 확인용
data = train_set_custom[7203ß]
print(data[1], data[0].shape)

ipd.Audio(data[0], rate=16000)

torch.Size([])
(16000,) torch.Size([1, 16000])
tensor(4) torch.Size([1, 16000])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in multiply


Data Loader

In [13]:
batch_size = 32
train_loader = DataLoader(train_set_custom, 
                          batch_size = batch_size, 
                          shuffle = True, 
                          pin_memory = True, 
                          num_workers = 2, 
                          worker_init_fn = lambda _: np.random.seed())

valid_loader = DataLoader(valid_set_custom, 
                          batch_size = batch_size, 
                          shuffle = False, 
                          pin_memory = True, 
                          num_workers = 2, 
                          worker_init_fn = lambda _: np.random.seed())

Train / Valid 함수

In [14]:
def number_of_correct(pred, target):
    return pred.squeeze().eq(target).sum().item()

def get_likely_index(tensor):
    return tensor.argmax(dim=-1)
    
def train(model, ep, opt, lossmod):
    model.train()
    losses = []
    for batch_idx, (wav, label) in enumerate(tqdm(train_loader, position = 0)):
        wav, label = wav.to(device), label.to(device)
        
        z = net(wav)
        loss = lossmod(z, label)
        opt.zero_grad()
        loss.backward()
        opt.step()

        losses.append(loss.item())
    print('Epoch: {} Loss: {:.4f}'.format(ep, np.average(losses)))

def validation(model, ep):
    model.eval()
    correct = 0
    losses = []
    for data, target in tqdm(valid_loader, position = 0):
        data = data.to(device)
        target = target.to(device)

        output = net(data)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)
    
    print("Valid epoch: {}, acc: {:.4f}".format(ep, correct / len(valid_set)))
# epoch별 publicset 점수 확인
def validation_by_publicset(net):
    with torch.no_grad():
      correct = 0
      net.eval()
      preds = []
      for i, wav_npy_dir in enumerate(tqdm(wav_npy_list, position = 0, leave = True)):
          wav_npy = np.load(wav_npy_dir)
          wav_npy = torch.tensor(np.reshape(wav_npy, (1, 1, wav_npy.shape[0]))).to(device)
          ans = torch.tensor(ans_npy).to(device)
          
          output = net(wav_npy)

          pred = get_likely_index(output)
          correct += number_of_correct(pred, ans[i])        

          preds.append(pred.detach().cpu().numpy()[0])
      print("publicset_ Acc: {:.4f}".format(correct / len(wav_npy_list)))
      print('----------------------------------------')


Model : 2D CNN

In [15]:
class ConvBlock2D(nn.Module):
    def __init__(self, c_i, c_o, k = 3, s = 1, pool_k = 2):
        super().__init__()
        self.block = nn.Sequential(nn.Conv2d(c_i, c_o, k, s),
                                   nn.BatchNorm2d(c_o),
                                   nn.ReLU(),
                                   nn.MaxPool2d(pool_k))
    def forward(self, x):
        return self.block(x)

class CNN2D(nn.Module):
    def __init__(self, c_in, c, num_classes):
        super().__init__()
        self.spec = torchaudio.transforms.MelSpectrogram(sample_rate = 16000, n_fft = 1024, hop_length=256, n_mels = 128)
        self.convnet = nn.Sequential(ConvBlock2D(c_in, c),
                                     ConvBlock2D(c, c),
                                     ConvBlock2D(c, 2 * c),
                                     ConvBlock2D(2 * c, 2 * c))
        self.linear = nn.Linear(2 * c, num_classes)
    
    def forward(self, x):
        x = torch.log10(self.spec(x) + 1e-5) # log-mel spectrogram: 1D audio -> 2D image-like representation
        z = self.convnet(x) # log-mel spec -> 2D CNN -> feature map C x F x T 
        z = rearrange(F.avg_pool2d(z, z.shape[-2:]), 'b c 1 1 -> b c') # feature map C x F x T -> feature C x 1 x 1
        z = self.linear(z) # feature C -> logit N
        return z

In [16]:
net = CNN2D(c_in = 1, c = 128, num_classes = len(labels)).to(device)
summary(net, (1, 1, 16000))

KeyboardInterrupt: 

Training : 2D CNN

In [ ]:
n_epoch = 5
opt = torch.optim.Adam(net.parameters(), lr = 1e-3)
ce = nn.CrossEntropyLoss()
for epoch in range(1, n_epoch + 1):
    train(net, epoch, opt, ce)
    validation(net, epoch)
    # epoch별 publicset 점수 확인
    validation_by_publicset(net)

100%|██████████| 2652/2652 [00:59<00:00, 44.56it/s]


Epoch: 1 Loss: 1.1679


100%|██████████| 312/312 [00:05<00:00, 53.69it/s]


Valid epoch: 1, acc: 0.8525


100%|██████████| 51/51 [00:00<00:00, 281.43it/s]


...

Valid epoch: 2, acc: 0.8778


100%|██████████| 51/51 [00:00<00:00, 271.95it/s]


Acc: 0.4902


 38%|███▊      | 1015/2652 [00:22<00:35, 45.96it/s]


KeyboardInterrupt: ignored

For Evaluation

In [ ]:
!ls drive/MyDrive/public_set/npy

0  1  2


In [ ]:
public_testset_dir = 'drive/MyDrive/public_set/npy'
folder = '1'

In [ ]:
wav_npy_list = glob.glob(os.path.join(public_testset_dir, folder, '*.npy'))
wav_npy_list.sort()
wav_npy_list = wav_npy_list[:-1]

ans_npy = np.load(os.path.join(public_testset_dir, folder, 'ans.npy'))

In [ ]:
with torch.no_grad():
    correct = 0
    net.eval()
    preds = []
    for i, wav_npy_dir in enumerate(tqdm(wav_npy_list, position = 0, leave = True)):
        wav_npy = np.load(wav_npy_dir)
        wav_npy = torch.tensor(np.reshape(wav_npy, (1, 1, wav_npy.shape[0]))).to(device)
        ans = torch.tensor(ans_npy).to(device)
        
        output = net(wav_npy)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, ans[i])        

        preds.append(pred.detach().cpu().numpy()[0])
    print("Acc: {:.4f}".format(correct / len(wav_npy_list)))

100%|██████████| 51/51 [00:19<00:00,  2.64it/s]

Acc: 0.4706


In [ ]:
pred.detach().cpu().numpy()[0]

26

Save predict.npy \
data shape을 꼭 확인하세요!

In [ ]:
# np.save('predict.npy', np.asarray(preds))